This is the first working notebook pipeline for the areal challenge solution

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

from tqdm import tqdm
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error

print("Imported Libraries Successfully")

In [ ]:
VERSION = "v3"

In [ ]:
# import os

# !ls -lh /kaggle/working

# import shutil

# # Compress your .npy file
# shutil.make_archive('/kaggle/working/signal_v2', 'zip', '/kaggle/working', 'signal_v2.npy')

# from IPython.display import FileLink
# FileLink('/kaggle/input/ariel-data-challenge-2025/train/1024292144/AIRS-CH0_signal_0.parquet')


In [ ]:
%%writefile preprocess.py

import pandas as pd
import numpy as np
import multiprocessing as mp
import itertools
import os
import subprocess
from astropy.stats import sigma_clip
from numpy.polynomial import Polynomial
from tqdm import tqdm
import torch
import torch.nn.functional as F


ROOT = "/kaggle/input/ariel-data-challenge-2025/"
VERSION = "v2"
A_BINNING = 15
F_BINNING = 12*15


device = ("cuda:0" if torch.cuda.is_available() else "cpu")
print("device : ", device)
MODE = os.getenv('PREPROCESS_MODE')


sensor_sizes_dict = {
    "AIRS-CH0": [[11250, 32, 356], [32, 356]],
    "FGS1": [[135000, 32, 32], [32, 32]],
}  # input, mask

cl = 8
cr = 24


def get_gain_offset():
    """
    Get the gain and offset for a given planet and sensor

    Unlike last year's challenge, all planets use the same adc_info.
    We can just hard code it.
    """
    gain = 0.4369
    offset = -1000.0
    return gain, offset



def read_data(planet_id, sensor, mode):
    """
    Read the data for a given planet and sensor
    """
    # get all noise correction frames and signal
    signal = pd.read_parquet(
        f"{ROOT}/{mode}/{planet_id}/{sensor}_signal_0.parquet",
        engine="pyarrow",
    )
    dark_frame = pd.read_parquet(
        f"{ROOT}/{mode}/{planet_id}/{sensor}_calibration_0/dark.parquet",
        engine="pyarrow",
    )
    dead_frame = pd.read_parquet(
        f"{ROOT}/{mode}/{planet_id}/{sensor}_calibration_0/dead.parquet",
        engine="pyarrow",
    )
    linear_corr_frame = pd.read_parquet(
        f"{ROOT}/{mode}/{planet_id}/{sensor}_calibration_0/linear_corr.parquet",
        engine="pyarrow",
    )
    flat_frame = pd.read_parquet(
        f"{ROOT}/{mode}/{planet_id}/{sensor}_calibration_0/flat.parquet",
        engine="pyarrow",
    )

    # reshape to sensor shape and cast to float64
    signal = signal.values.astype(np.float64).reshape(sensor_sizes_dict[sensor][0])[
        :, cl:cr, :
    ]
    dark_frame = dark_frame.values.astype(np.float64).reshape(
        sensor_sizes_dict[sensor][1]
    )[cl:cr, :]
    dead_frame = dead_frame.values.reshape(sensor_sizes_dict[sensor][1])[cl:cr, :]
    flat_frame = flat_frame.values.astype(np.float64).reshape(
        sensor_sizes_dict[sensor][1]
    )[cl:cr, :]

    linear_corr = linear_corr_frame.values.astype(np.float64).reshape(
        [6] + sensor_sizes_dict[sensor][1]
    )[:, cl:cr, :]

    return (
        signal,
        dark_frame,
        dead_frame,
        linear_corr,
        flat_frame,
    )


def ADC_convert(signal, gain, offset):
    """
    Step 1: Analog-to-Digital Conversion (ADC) correction

    The Analog-to-Digital Conversion (adc) is performed by the detector to convert the
    pixel voltage into an integer number. We revert this operation by using the gain
    and offset for the calibration files 'train_adc_info.csv'.
    """

    signal /= gain
    signal += offset
    return signal


def mask_hot_dead(signal, dead, dark):
    """
    Step 2: Mask hot/dead pixel

    The dead pixels map is a map of the pixels that do not respond to light and, thus,
    can't be accounted for any calculation. In all these frames the dead pixels are
    masked using python masked arrays. The bad pixels are thus masked but left
    uncorrected. Some methods can be used to correct bad-pixels but this task,
    if needed, is left to the participants.
    """

    hot = sigma_clip(dark, sigma=5, maxiters=5).mask
    hot = np.tile(hot, (signal.shape[0], 1, 1))
    dead = np.tile(dead, (signal.shape[0], 1, 1))

    signal[dead] = np.nan
    signal[hot] = np.nan
    return signal


def apply_linear_corr(c, signal):
    """
    Step 3: linearity Correction

    The non-linearity of the pixels' response can be explained as capacitive leakage
    on the readout electronics of each pixel during the integration time. The number
    of electrons in the well is proportional to the number of photons that hit the
    pixel, with a quantum efficiency coefficient. However, the response of the pixel
    is not linear with the number of electrons in the well. This effect can be
    described by a polynomial function of the number of electrons actually in the well.
    The data is provided with calibration files linear_corr.parquet that are the
    coefficients of the inverse polynomial function and can be used to correct this
    non-linearity effect.
    Using horner's method to evaluate the polynomial
    """
    assert c.shape[0] == 6  # Ensure the polynomial is of degree 5

    return (
        (((c[5] * signal + c[4]) * signal + c[3]) * signal + c[2]) * signal + c[1]
    ) * signal + c[0]


def clean_dark(signal, dark, dt):
    """
    Step 4: dark current subtraction

    The data provided include calibration for dark current estimation, which can be
    used to pre-process the observations. Dark current represents a constant signal
    that accumulates in each pixel during the integration time, independent of the
    incoming light. To obtain the corrected image, the following conventional approach
    is applied: The data provided include calibration files such as dark frames or
    dead pixels' maps. They can be used to pre-process the observations. The dark frame
    is a map of the detector response to a very short exposure time, to correct for the
    dark current of the detector.

    image - (dark * dt)

    The corrected image is conventionally obtained via the following: where the dark
    current map is first corrected for the dead pixel.
    """

    dark = torch.tile(dark, (signal.shape[0], 1, 1))
    signal -= dark * dt[:, None, None]
    return signal


def get_cds(signal):
    """
    Step 5: Get Correlated Double Sampling (CDS)

    The science frames are alternating between the start of the exposure and the end of
    the exposure. The lecture scheme is a ramp with a double sampling, called
    Correlated Double Sampling (CDS), the detector is read twice, once at the start
    of the exposure and once at the end of the exposure. The final CDS is the
    difference (End of exposure) - (Start of exposure).
    """

    return torch.subtract(signal[1::2, :, :], signal[::2, :, :])


def correct_flat_field(flat, signal):
    """
    Step 6: Flat Field Correction

    The flat field is a map of the detector response to uniform illumination, to
    correct for the pixel-to-pixel variations of the detector, for example the
    different quantum efficiencies of each pixel.
    """

    return signal / flat


def bin_obs(signal, binning):
    """
    Step 5.1: Bin Observations

    The data provided are binned in the time dimension. The binning is performed by
    summing the signal over the time dimension.
    """

    cds_binned = torch.zeros(
        (
            signal.shape[0] // binning,
            signal.shape[1],
            signal.shape[2],
        ),
        device=device,
    )
    for i in range(signal.shape[0] // binning):
        cds_binned[i, :, :] = torch.sum(
            signal[i * binning : (i + 1) * binning, :, :], axis=0
        )
    return cds_binned


def nan_interpolation(tensor):
    # Assume tensor is of shape (batch, height, width)
    nan_mask = torch.isnan(tensor)

    # Replace NaNs with zero temporarily
    tensor_filled = torch.where(
        nan_mask, torch.tensor(0.0, device=tensor.device), tensor
    )

    # Create a binary mask (0 where NaNs were and 1 elsewhere)
    ones = torch.ones_like(tensor, device=tensor.device)
    weight = torch.where(nan_mask, torch.tensor(0.0, device=tensor.device), ones)

    # Perform interpolation by convolving with a kernel
    # using bilinear interpolation
    kernel = torch.ones(1, 1, 1, 3, device=tensor.device, dtype=tensor.dtype)

    # Apply padding to the tensor and weight to prevent boundary issues
    tensor_padded = F.pad(
        tensor_filled.unsqueeze(1), (1, 1, 0, 0), mode="replicate"
    ).squeeze(1)
    weight_padded = F.pad(weight.unsqueeze(1), (1, 1, 0, 0), mode="replicate").squeeze(
        1
    )

    # Convolve the filled tensor and the weight mask
    tensor_conv = F.conv2d(tensor_padded.unsqueeze(1), kernel, stride=1)
    weight_conv = F.conv2d(weight_padded.unsqueeze(1), kernel, stride=1)

    # Compute interpolated values (normalized by weights)
    interpolated_tensor = tensor_conv / weight_conv

    # Apply the interpolated values only to the positions of NaNs
    result = torch.where(nan_mask, interpolated_tensor.squeeze(1), tensor)

    return result



def process_planet(planet_id):
    """
    Process a single planet's data
    """
    axis_info = pd.read_parquet(ROOT + "/axis_info.parquet")
    dt_airs = axis_info["AIRS-CH0-integration_time"].dropna().values

    for sensor in ["AIRS-CH0", "FGS1"]:
        # load all data for this planet and sensor
        signal, dark_frame, dead_frame, linear_corr, flat_frame = read_data(
            planet_id, sensor, mode=MODE
        )
        gain, offset = get_gain_offset()

        # Step 1: ADC correction
        signal = ADC_convert(signal, gain, offset)

        # Step 2: Mask hot/dead pixel
        signal = mask_hot_dead(signal, dead_frame, dark_frame)

        # clip at 0
        signal = torch.tensor(signal.clip(0)).to(device)    
        signal = apply_linear_corr(
            torch.tensor(linear_corr).to(device), signal.clone().detach()
        )

        if sensor == "FGS1":
            dt = torch.ones(len(signal), device=device) * 0.1
        elif sensor == "AIRS-CH0":
            dt = torch.tensor(dt_airs).to(device)

        dt[1::2] += 0.1

        signal = clean_dark(signal, torch.tensor(dark_frame).to(device), dt)

        # Step 5: Get Correlated Double Sampling (CDS)
        signal = get_cds(signal)

        # Step 6: Flat Field Correction

        if sensor == "AIRS-CH0":
            signal = bin_obs(signal, binning=A_BINNING)
        else:
            signal = bin_obs(signal, binning=F_BINNING)

        signal = correct_flat_field(torch.tensor(flat_frame).to(device), signal)

        # Step 7: Interpolate NaNs (twice!)
        signal = nan_interpolation(signal)
        signal = nan_interpolation(signal)

        if sensor == "FGS1":
            signal = torch.nanmean(signal, axis=[1, 2]).cpu().numpy()
        elif sensor == "AIRS-CH0":
            signal = torch.nanmean(signal, axis=1).cpu().numpy()
            
        # save the processed signal
        np.save(
            str(planet_id) + "_" + sensor + f"_signal_{VERSION}.npy",
            signal.astype(np.float64),
        )

if __name__ == "__main__":
    star_info = pd.read_csv(ROOT + f"/{MODE}_star_info.csv", index_col="planet_id")
    planet_ids = [int(x) for x in star_info.index.tolist()]

    # Use up to 4 threads!
    mp.set_start_method('spawn')
    with mp.Pool(processes=4) as pool:
        list(tqdm(pool.imap(process_planet, planet_ids), total=len(planet_ids)))

    
    signal_train = []

    for planet_id in planet_ids:
        f_raw = np.load(f"{planet_id}_FGS1_signal_{VERSION}.npy")
        a_raw = np.load(f"{planet_id}_AIRS-CH0_signal_{VERSION}.npy")

        # flip a_raw
        signal = np.concatenate([f_raw[:, None], a_raw[:, ::-1]], axis=1)
        signal_train.append(signal)

        os.remove("/kaggle/working/" + str(planet_id) + f"_AIRS-CH0_signal_{VERSION}.npy")
        os.remove("/kaggle/working/" + str(planet_id) + f"_FGS1_signal_{VERSION}.npy")

    signal_train = np.array(signal_train)
    np.save(f"signal_{VERSION}.npy", signal_train, allow_pickle=False)

    print("Processing complete!")

In [ ]:
data_path = f"/kaggle/input/signal-v2-1-npy/signal_{VERSION} (1).npy"
os.environ["PREPROCESS_MODE"] = "train"

# Check if preprocessed data already exists
if not os.path.exists(data_path):
    print("Preprocessed data not found. Running preprocessing...")

    # Set environment variable for your script
    os.environ["PREPROCESS_MODE"] = "train"

    # Run the preprocessing script
    !python preprocess.py

    # (Assuming preprocess.py saves 'signal_{VERSION}.npy')
else:
    print("Preprocessed data already exists. Skipping preprocessing.")

In [ ]:
data_train = np.load(f"/kaggle/working/signal_v2.npy")
data_train.shape

In [ ]:
print("check, done")

In [ ]:
%%writefile view_signal.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats, ndimage
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('dark_background')
sns.set_palette("viridis")

# =============================================================================
# CONFIGURATION
# =============================================================================

CONFIG = {
    # Data paths
    'base_path': "/kaggle/input/ariel-data-challenge-2025",
    'data_split': "test",  # "train" or "test"
    
    # Target selection
    'target_id': "1103775",  # Specific target ID or None for first available
    
    # Instruments to analyze
    'instruments': ['FGS1', 'AIRS-CH0'],
    
    # Calibration types to analyze
    'calibration_types': ['dark', 'flat', 'linear_corr', 'dead', 'read'],
    
    # Analysis options
    'analyze_calibration': True,
    'analyze_science': True,
    'analyze_atmosphere': True,
    
    # Atmospheric analysis parameters
    'spectral_rows': (10, 22),  # Rows to extract spectrum from AIRS-CH0
    'out_of_transit_frames': (0, 10),  # Frame range for baseline
    'in_transit_frames': (20, 30),  # Frame range for transit
    
    # Detector specifications
    'detector_specs': {
        'FGS1': {
            'shape': (32, 32),
            'band': 'Visible (0.5-0.8 μm)',
            'purpose': 'Fine Guidance',
            'cmap': 'magma',
            'wavelength_range': None
        },
        'AIRS-CH0': {
            'shape': (32, 356),
            'band': 'NIR (0.95-1.95 μm)',
            'purpose': 'Spectroscopy',
            'cmap': 'inferno',
            'wavelength_range': (0.95, 1.95)
        }
    },
    
    # Molecular absorption bands (for atmospheric analysis)
    'molecular_bands': {
        'H2O': [1.1, 1.2, 1.35, 1.5, 1.87],
        'CH4': [1.6, 1.8]
    },
    
    # Visualization options
    'verbose': False,  # Set to True for debug output
    'show_plots': True
}

# =============================================================================
# CALIBRATION DATA ANALYSIS
# =============================================================================

class CalibrationAnalyzer:
    def __init__(self, config):
        self.config = config
        self.base_path = config['base_path']
        self.data_split = config['data_split']
        self.detector_specs = config['detector_specs']
    
    def load_calibration_data(self, target_id, instrument, cal_type):
        """Load specific calibration data"""
        cal_path = Path(self.base_path) / self.data_split / target_id / \
                   f"{instrument}_calibration_0" / f"{cal_type}.parquet"
        
        if cal_path.exists():
            return pd.read_parquet(cal_path)
        return None
    
    def analyze_calibration_frame(self, data, detector_shape):
        """Analyze a single calibration frame"""
        if data is None or data.empty:
            return None, None
        
        try:
            # Handle various data formats
            if data.shape == detector_shape:
                frame = data.values
            elif len(data.shape) == 2 and data.shape[1:] == detector_shape:
                frame = data.iloc[0].values if hasattr(data, 'iloc') else data[0]
            elif len(data.shape) == 2 and data.shape[0] > detector_shape[0]:
                frames_per_stack = data.shape[0] // detector_shape[0]
                frame = data.values.reshape(frames_per_stack, *detector_shape).mean(axis=0)
            elif data.shape[1] == np.prod(detector_shape):
                frame = data.iloc[0].values.reshape(detector_shape)
            else:
                frame = data.values
            
            # Handle boolean data (dead pixel masks)
            if frame.dtype == bool:
                frame = frame.astype(int)
            
            frame = np.array(frame, dtype=float)
            
            stats_dict = {
                'mean': np.mean(frame),
                'median': np.median(frame),
                'std': np.std(frame),
                'min': np.min(frame),
                'max': np.max(frame),
                'hot_pixels': np.sum(frame > np.percentile(frame, 99.9)) if frame.max() > frame.min() else 0,
                'dead_pixels': np.sum(frame == 0),
                'shape': frame.shape
            }
            
            return frame, stats_dict
            
        except Exception as e:
            if self.config['verbose']:
                print(f"Error processing calibration frame: {e}")
            return None, None
    
    def plot_calibration_suite(self, target_id, instrument):
        """Comprehensive calibration analysis for one instrument"""
        cal_types = self.config['calibration_types']
        detector_shape = self.detector_specs[instrument]['shape']
        band_info = self.detector_specs[instrument]['band']
        
        fig, axes = plt.subplots(2, 3, figsize=(20, 12))
        fig.suptitle(f'🛸 {instrument} Calibration Suite - Target {target_id}\n{band_info}', 
                    fontsize=16, y=0.98)
        
        axes = axes.flatten()
        
        for i, cal_type in enumerate(cal_types):
            ax = axes[i]
            cal_data = self.load_calibration_data(target_id, instrument, cal_type)
            
            if cal_data is not None and not cal_data.empty:
                frame, stats = self.analyze_calibration_frame(cal_data, detector_shape)
                
                if frame is not None:
                    cmap = {
                        'dark': 'inferno',
                        'flat': 'plasma',
                        'linear_corr': 'viridis',
                        'dead': 'binary',
                        'read': 'magma'
                    }.get(cal_type, 'viridis')
                    
                    im = ax.imshow(frame, cmap=cmap, aspect='auto')
                    plt.colorbar(im, ax=ax, fraction=0.046)
                    
                    stats_text = f"μ={stats['mean']:.1f}\nσ={stats['std']:.1f}\nHot:{stats['hot_pixels']}\nDead:{stats['dead_pixels']}"
                    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
                           bbox=dict(boxstyle="round,pad=0.3", facecolor="black", alpha=0.8),
                           verticalalignment='top', fontsize=9, color='white')
                else:
                    ax.text(0.5, 0.5, f"❌ Processing Error", ha='center', va='center', 
                           transform=ax.transAxes, fontsize=12, color='red')
            else:
                ax.text(0.5, 0.5, f"❌ No Data", ha='center', va='center', 
                       transform=ax.transAxes, fontsize=12, color='red')
            
            ax.set_title(f"{cal_type.upper()} Calibration", fontsize=12)
            ax.set_xlabel("Pixel Column")
            ax.set_ylabel("Pixel Row")
        
        if len(cal_types) < len(axes):
            axes[-1].remove()
        
        plt.tight_layout()
        if self.config['show_plots']:
            plt.show()
        
        return True

# =============================================================================
# SCIENCE DATA VISUALIZATION
# =============================================================================

class ScienceDataVisualizer:
    def __init__(self, config):
        self.config = config
        self.base_path = config['base_path']
        self.data_split = config['data_split']
        self.detector_specs = config['detector_specs']
    
    def load_science_data(self, target_id, instrument):
        """Load science observation data"""
        sci_path = Path(self.base_path) / self.data_split / target_id / \
                   f"{instrument}_signal_0.parquet"
        
        if sci_path.exists():
            return pd.read_parquet(sci_path)
        return None
    
    def analyze_time_series(self, data, detector_shape):
        """Analyze temporal behavior of observations"""
        if data is None or data.empty:
            return None, None
        
        try:
            expected_pixels = np.prod(detector_shape)
            
            if data.shape[1] == expected_pixels:
                flux_series = data.sum(axis=1).values
                
                frame_stats = []
                num_frames = min(len(data), 50)
                
                for i in range(num_frames):
                    frame = data.iloc[i].values.reshape(detector_shape)
                    frame_stats.append({
                        'frame': i,
                        'total_flux': np.sum(frame),
                        'mean_flux': np.mean(frame),
                        'max_flux': np.max(frame),
                        'std_flux': np.std(frame)
                    })
                
                stats_df = pd.DataFrame(frame_stats)
                return flux_series, stats_df
            else:
                return None, None
                
        except Exception as e:
            if self.config['verbose']:
                print(f"Error in time series analysis: {e}")
            return None, None
    
    def plot_exoplanet_analysis(self, target_id):
        """Comprehensive exoplanet observation analysis"""
        fig = plt.figure(figsize=(24, 16))
        gs = fig.add_gridspec(4, 4, hspace=0.3, wspace=0.3)
        
        fig.suptitle(f'🌌 Exoplanet Observation Analysis - Target {target_id}', 
                    fontsize=18, y=0.98)
        
        instruments = self.config['instruments']
        
        for idx, instrument in enumerate(instruments):
            data = self.load_science_data(target_id, instrument)
            if data is None:
                continue
            
            detector_shape = self.detector_specs[instrument]['shape']
            cmap = self.detector_specs[instrument]['cmap']
            band = self.detector_specs[instrument]['band']
            
            # 1. First frame
            ax1 = fig.add_subplot(gs[idx*2, 0])
            try:
                frame0 = data.iloc[0].values.reshape(detector_shape)
                im1 = ax1.imshow(frame0, cmap=cmap, aspect='auto')
                plt.colorbar(im1, ax=ax1, fraction=0.046)
                ax1.set_title(f"{instrument} - First Frame\n{band}", fontsize=11)
            except Exception as e:
                ax1.text(0.5, 0.5, f"❌ Error", ha='center', va='center', 
                        transform=ax1.transAxes, fontsize=10)
            
            # 2. Last frame
            ax2 = fig.add_subplot(gs[idx*2, 1])
            try:
                frame_last = data.iloc[-1].values.reshape(detector_shape)
                im2 = ax2.imshow(frame_last, cmap=cmap, aspect='auto')
                plt.colorbar(im2, ax=ax2, fraction=0.046)
                ax2.set_title(f"{instrument} - Last Frame", fontsize=11)
            except Exception as e:
                ax2.text(0.5, 0.5, f"❌ Error", ha='center', va='center', 
                        transform=ax2.transAxes, fontsize=10)
            
            # 3. Difference frame
            ax3 = fig.add_subplot(gs[idx*2, 2])
            try:
                if 'frame0' in locals() and 'frame_last' in locals():
                    diff_frame = frame_last - frame0
                    im3 = ax3.imshow(diff_frame, cmap='RdBu_r', aspect='auto')
                    plt.colorbar(im3, ax=ax3, fraction=0.046)
                    ax3.set_title(f"{instrument} - Difference\n(Last - First)", fontsize=11)
            except Exception as e:
                ax3.text(0.5, 0.5, f"❌ Error", ha='center', va='center', 
                        transform=ax3.transAxes, fontsize=10)
            
            # 4. Time series
            flux_series, stats_df = self.analyze_time_series(data, detector_shape)
            
            if flux_series is not None:
                ax4 = fig.add_subplot(gs[idx*2:(idx*2)+2, 3])
                time_points = np.arange(len(flux_series))
                ax4.plot(time_points, flux_series, 'o-', alpha=0.7, linewidth=2, markersize=3)
                
                if len(flux_series) > 10:
                    poly_fit = np.polyfit(time_points, flux_series, 3)
                    smooth_flux = np.polyval(poly_fit, time_points)
                    ax4.plot(time_points, smooth_flux, 'r--', alpha=0.8, linewidth=2, 
                            label='Polynomial Fit')
                    ax4.legend()
                
                ax4.set_title(f"{instrument} - Flux Time Series", fontsize=11)
                ax4.set_xlabel("Frame Number")
                ax4.set_ylabel("Total Flux (counts)")
                ax4.grid(True, alpha=0.3)
                
                flux_normalized = flux_series / np.median(flux_series)
                min_flux = np.min(flux_normalized)
                if min_flux < 0.998:
                    transit_depth = (1 - min_flux) * 100
                    ax4.text(0.02, 0.98, f"Transit Depth: {transit_depth:.3f}%", 
                            transform=ax4.transAxes, bbox=dict(boxstyle="round,pad=0.3", 
                            facecolor="yellow", alpha=0.8), verticalalignment='top')
            
            # 5. Mean frame
            ax5 = fig.add_subplot(gs[idx*2+1, 0])
            mean_frame = data.mean(axis=0).values.reshape(detector_shape)
            im5 = ax5.imshow(mean_frame, cmap=cmap, aspect='auto')
            plt.colorbar(im5, ax=ax5, fraction=0.046)
            ax5.set_title(f"{instrument} - Mean Frame", fontsize=11)
            
            # 6. Noise map
            ax6 = fig.add_subplot(gs[idx*2+1, 1])
            std_frame = data.std(axis=0).values.reshape(detector_shape)
            im6 = ax6.imshow(std_frame, cmap='hot', aspect='auto')
            plt.colorbar(im6, ax=ax6, fraction=0.046)
            ax6.set_title(f"{instrument} - Noise Map (σ)", fontsize=11)
            
            # 7. SNR map
            ax7 = fig.add_subplot(gs[idx*2+1, 2])
            snr_frame = np.divide(mean_frame, std_frame, 
                                out=np.zeros_like(mean_frame), where=std_frame!=0)
            im7 = ax7.imshow(snr_frame, cmap='plasma', aspect='auto', vmin=0, vmax=50)
            plt.colorbar(im7, ax=ax7, fraction=0.046)
            ax7.set_title(f"{instrument} - SNR Map", fontsize=11)
        
        plt.tight_layout()
        if self.config['show_plots']:
            plt.show()
        
        return True

# =============================================================================
# ATMOSPHERIC COMPOSITION ANALYSIS
# =============================================================================

class AtmosphericAnalyzer:
    def __init__(self, config):
        self.config = config
        self.base_path = config['base_path']
        self.data_split = config['data_split']
        self.sci_viz = ScienceDataVisualizer(config)
    
    def analyze_composition(self, target_id):
        """Extract atmospheric composition from transit spectroscopy"""
        
        airs_data = self.sci_viz.load_science_data(target_id, 'AIRS-CH0')
        
        if airs_data is None:
            if self.config['verbose']:
                print("No AIRS-CH0 data available for atmospheric analysis")
            return None, None, None
        
        # Extract spectral configuration
        spectral_rows = self.config['spectral_rows']
        wl_range = self.config['detector_specs']['AIRS-CH0']['wavelength_range']
        
        # Reshape to (time, wavelength)
        spectra = []
        for i in range(len(airs_data)):
            frame = airs_data.iloc[i].values.reshape(32, 356)
            spectrum = np.sum(frame[spectral_rows[0]:spectral_rows[1], :], axis=0)
            spectra.append(spectrum)
        
        spectra = np.array(spectra)
        time_points = np.arange(len(spectra))
        wavelengths = np.linspace(wl_range[0], wl_range[1], 356)
        
        self._plot_atmospheric_analysis(target_id, wavelengths, spectra, time_points)
        
        return wavelengths, spectra, time_points
    
    def _plot_atmospheric_analysis(self, target_id, wavelengths, spectra, time_points):
        """Generate atmospheric composition plots"""
        
        fig, axes = plt.subplots(2, 2, figsize=(20, 12))
        fig.suptitle(f'🌍 Atmospheric Composition Analysis - Target {target_id}', fontsize=16)
        
        # 1. Transit light curves at key wavelengths
        ax1 = axes[0, 0]
        key_wavelengths = [50, 100, 150, 200, 250, 300]
        colors = plt.cm.viridis(np.linspace(0, 1, len(key_wavelengths)))
        
        for wl_idx, color in zip(key_wavelengths, colors):
            normalized_flux = spectra[:, wl_idx] / np.median(spectra[:, wl_idx])
            ax1.plot(time_points, normalized_flux, 'o-', color=color, 
                    label=f'{wavelengths[wl_idx]:.2f} μm', alpha=0.8, markersize=3)
        
        ax1.set_xlabel('Frame Number')
        ax1.set_ylabel('Normalized Flux')
        ax1.set_title('Transit Depth vs Wavelength')
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax1.grid(True, alpha=0.3)
        
        # 2. Transmission spectrum
        ax2 = axes[0, 1]
        transit_depths = []
        for wl_idx in range(356):
            flux = spectra[:, wl_idx]
            normalized = flux / np.median(flux)
            min_flux = np.min(normalized)
            transit_depth = (1 - min_flux) * 100
            transit_depths.append(transit_depth)
        
        ax2.plot(wavelengths, transit_depths, 'b-', linewidth=2)
        ax2.set_xlabel('Wavelength (μm)')
        ax2.set_ylabel('Transit Depth (%)')
        ax2.set_title('Transmission Spectrum')
        ax2.grid(True, alpha=0.3)
        
        # Highlight molecular bands
        molecular_bands = self.config['molecular_bands']
        if 'H2O' in molecular_bands:
            for wl in molecular_bands['H2O'][:2]:
                if wavelengths[0] <= wl <= wavelengths[-1]:
                    ax2.axvline(wl, alpha=0.3, color='blue', linestyle='--')
        
        if 'CH4' in molecular_bands:
            for wl in molecular_bands['CH4']:
                if wavelengths[0] <= wl <= wavelengths[-1]:
                    ax2.axvline(wl, alpha=0.3, color='red', linestyle='--')
        
        # 3. Spectral evolution
        ax3 = axes[1, 0]
        im = ax3.imshow(spectra.T, aspect='auto', origin='lower', 
                        extent=[0, len(time_points), wavelengths[0], wavelengths[-1]],
                        cmap='plasma')
        plt.colorbar(im, ax=ax3, label='Flux (counts)')
        ax3.set_xlabel('Frame Number')
        ax3.set_ylabel('Wavelength (μm)')
        ax3.set_title('Spectral Evolution During Transit')
        
        # 4. Relative absorption
        ax4 = axes[1, 1]
        out_range = self.config['out_of_transit_frames']
        in_range = self.config['in_transit_frames']
        
        out_of_transit = np.mean(spectra[out_range[0]:out_range[1]], axis=0)
        in_transit = np.mean(spectra[in_range[0]:in_range[1]], axis=0)
        
        relative_absorption = (out_of_transit - in_transit) / out_of_transit * 100
        
        ax4.plot(wavelengths, relative_absorption, 'g-', linewidth=2)
        ax4.set_xlabel('Wavelength (μm)')
        ax4.set_ylabel('Relative Absorption (%)')
        ax4.set_title('Atmospheric Absorption Features')
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        if self.config['show_plots']:
            plt.show()

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def run_analysis(config=None):
    """Main analysis pipeline"""
    
    if config is None:
        config = CONFIG
    
    # Get target ID
    data_path = Path(config['base_path']) / config['data_split']
    
    if not data_path.exists():
        print(f"❌ Path not found: {data_path}")
        return
    
    targets = [d.name for d in data_path.iterdir() if d.is_dir()]
    
    if not targets:
        print(f"❌ No targets found in {data_path}")
        return
    
    target_id = config['target_id']
    if target_id is None:
        target_id = targets[0]
    elif target_id not in targets:
        print(f"❌ Target {target_id} not found. Using first available: {targets[0]}")
        target_id = targets[0]
    
    print(f"🎯 Analyzing Target: {target_id} from {config['data_split']} split")
    
    # Initialize analyzers
    cal_analyzer = CalibrationAnalyzer(config)
    sci_visualizer = ScienceDataVisualizer(config)
    atm_analyzer = AtmosphericAnalyzer(config)
    
    # Run analyses
    if config['analyze_calibration']:
        for instrument in config['instruments']:
            cal_analyzer.plot_calibration_suite(target_id, instrument)
    
    if config['analyze_science']:
        sci_visualizer.plot_exoplanet_analysis(target_id)
    
    if config['analyze_atmosphere']:
        atm_analyzer.analyze_composition(target_id)
    
    print("✅ Analysis Complete!")

# =============================================================================
# RUN
# =============================================================================

# if __name__ == "__main__":
#     run_analysis(CONFIG)

In [ ]:
exec(open('view_signal.py', 'r').read())
CONFIG['base_path'] = '/kaggle/input/ariel-data-challenge-2025'
CONFIG['data_split'] = 'train'   # or 'test' depending on stage
CONFIG['target_id'] = '1010375142'  # update to your target folder name
CONFIG['show_plots'] = True

In [ ]:
run_analysis(CONFIG)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------

DATA_ROOT = "/kaggle/input/ariel-data-challenge-2025"
SIGNAL_FILE = f"/kaggle/working/signal_v2.npy"   # preprocessed data file (all planets)
STAR_INFO = f"{DATA_ROOT}/train_star_info.csv"

# ---------------------------------------------------------
# LOAD UTILITIES
# ---------------------------------------------------------

def load_preprocessed_signal(signal_path=SIGNAL_FILE):
    """Load preprocessed (npy) signal file"""
    signal = np.load(signal_path)
    print(f"✅ Loaded preprocessed signal: shape {signal.shape}")
    return signal

def load_raw_signal(planet_id, base_path=DATA_ROOT, split="train"):
    """Load raw parquet signals for one planet"""
    base = Path(base_path) / split / str(planet_id)
    fgs1_path = base / "FGS1_signal_0.parquet"
    airs_path = base / "AIRS-CH0_signal_0.parquet"
    if not fgs1_path.exists():
        raise FileNotFoundError(f"No data found for {planet_id}")

    fgs1 = pd.read_parquet(fgs1_path).values
    airs = pd.read_parquet(airs_path).values

    # Collapse FGS1 to 1D light curve (sum of all detector pixels)
    fgs1 = fgs1.sum(axis=1)
    return fgs1, airs

# ---------------------------------------------------------
# VISUALIZATION
# ---------------------------------------------------------
def plot_signals(fgs1, airs, title_prefix="Planet"):
    """Visualize FGS1 (1D) and AIRS-CH0 (1D or 2D) signals"""
    import matplotlib.pyplot as plt

    # --- Create subplots dynamically ---
    if airs.ndim == 1:
        fig, ax = plt.subplots(1, 2, figsize=(14, 4))
    else:
        fig, ax = plt.subplots(1, 2, figsize=(14, 5))

    # --- 1️⃣ FGS1 Light Curve ---
    ax[0].plot(fgs1, color='cyan', lw=1.5)
    ax[0].set_title(f"{title_prefix} — FGS1 Light Curve", fontsize=12)
    ax[0].set_xlabel("Time bin")
    ax[0].set_ylabel("Flux (counts)")
    ax[0].grid(alpha=0.3)

    # --- 2️⃣ AIRS Visualization ---
    if airs.ndim == 2:
        im = ax[1].imshow(airs.T, aspect='auto', origin='lower', cmap='inferno')
        ax[1].set_title(f"{title_prefix} — AIRS-CH0 Spectrum", fontsize=12)
        ax[1].set_xlabel("Time bin")
        ax[1].set_ylabel("Wavelength bin")
        plt.colorbar(im, ax=ax[1], fraction=0.046, label="Flux Intensity")
    elif airs.ndim == 1:
        ax[1].plot(airs, color='orange', lw=1.5)
        ax[1].set_title(f"{title_prefix} — AIRS-CH0 Light Curve", fontsize=12)
        ax[1].set_xlabel("Time bin")
        ax[1].set_ylabel("Flux (counts)")
        ax[1].grid(alpha=0.3)
    else:
        raise ValueError(f"Unexpected AIRS shape: {airs.shape}")

    plt.tight_layout()
    plt.show()

# ---------------------------------------------------------
# MAIN FUNCTION
# ---------------------------------------------------------

def visualize_planet(planet_idx=0, source="preprocessed"):
    """
    Visualize one planet's signals.
    source = 'preprocessed' or 'raw'
    """
    if source == "preprocessed":
        # Load npy file and get by index
        signal = load_preprocessed_signal()
        fgs1, airs = signal[planet_idx, 0], signal[planet_idx, 1]
        title_prefix = f"Preprocessed Planet {planet_idx}"

    elif source == "raw":
        # Map index to planet_id using star_info
        star_info = pd.read_csv(STAR_INFO, index_col="planet_id")
        planet_ids = star_info.index.to_list()
        planet_id = planet_ids[planet_idx]

        fgs1, airs = load_raw_signal(planet_id)
        title_prefix = f"Raw Planet ID {planet_id} (idx={planet_idx})"

    else:
        raise ValueError("source must be 'preprocessed' or 'raw'")

    plot_signals(fgs1, airs, title_prefix)



def compare_planet_signals(planet_idx, signal_v2_path=SIGNAL_FILE, raw_base_dir="/kaggle/input/ariel-data-challenge-2025/train"):
    """
    Compare raw and preprocessed signals for a given planet index.
    Automatically handles 1D vs 2D AIRS data.
    """

    # --- Load preprocessed data ---
    signal = np.load(signal_v2_path, allow_pickle=True)
    prep_fgs1, prep_airs = signal[planet_idx, 0], signal[planet_idx, 1]

    # --- Identify planet ID from folder structure ---
    planet_id = sorted(os.listdir(raw_base_dir))[planet_idx]
    print(f"🪐 Viewing Planet ID: {planet_id}")

    # --- Load raw data ---
    raw_fgs1_path = os.path.join(raw_base_dir, planet_id, "FGS1_signal_0.parquet")
    raw_airs_path = os.path.join(raw_base_dir, planet_id, "AIRS-CH0_signal_0.parquet")

    raw_fgs1 = pd.read_parquet(raw_fgs1_path).values
    raw_airs = pd.read_parquet(raw_airs_path).values

    # Flatten if needed
    if raw_fgs1.ndim > 1:
        raw_fgs1 = raw_fgs1.flatten()

    # --- Plot FGS1 (1D) ---
    plt.figure(figsize=(10, 4))
    plt.plot(raw_fgs1, label="Raw FGS1", alpha=0.6, color="orange")
    plt.plot(prep_fgs1, label="Preprocessed FGS1", alpha=0.8, color="cyan")
    plt.title(f"FGS1 Light Curve Comparison — Planet {planet_id}")
    plt.xlabel("Time bins")
    plt.ylabel("Flux")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

    # --- Plot AIRS (handles 1D or 2D automatically) ---
    if prep_airs.ndim == 1:
        fig, ax = plt.subplots(1, 1, figsize=(10, 4))
        mid_band = raw_airs.shape[1] // 2

        ax.plot(raw_airs[:, mid_band], label='Raw Central Band', alpha=0.6, color='orange')
        ax.plot(prep_airs, label='Preprocessed (Averaged)', alpha=0.8, color='cyan')
        ax.set_title(f"AIRS-CH0 Comparison — Planet {planet_id}")
        ax.set_xlabel("Time bin")
        ax.set_ylabel("Flux")
        ax.legend()
        ax.grid(alpha=0.3)
        plt.show()
    else:
        fig, ax = plt.subplots(1, 3, figsize=(15, 4))
        im0 = ax[0].imshow(raw_airs.T, aspect='auto', origin='lower', cmap='magma')
        ax[0].set_title("Raw AIRS-CH0")
        plt.colorbar(im0, ax=ax[0], fraction=0.046, pad=0.04)

        im1 = ax[1].imshow(prep_airs.T, aspect='auto', origin='lower', cmap='magma')
        ax[1].set_title("Preprocessed AIRS-CH0")
        plt.colorbar(im1, ax=ax[1], fraction=0.046, pad=0.04)

        mid_band = raw_airs.shape[1] // 2
        ax[2].plot(raw_airs[:, mid_band], label='Raw', alpha=0.6, color='orange')
        ax[2].plot(prep_airs[:, mid_band], label='Preprocessed', alpha=0.8, color='cyan')
        ax[2].set_title("Central Band Comparison")
        ax[2].set_xlabel("Time bin")
        ax[2].set_ylabel("Flux")
        ax[2].legend()
        ax[2].grid(alpha=0.3)

        fig.suptitle(f"AIRS-CH0 Comparison — Planet {planet_id}", fontsize=13)
        plt.tight_layout()
        plt.show()

def plot_fgs1_comparison(raw_fgs1, prep_fgs1, planet_id):
    import matplotlib.pyplot as plt
    import numpy as np

    # Normalize preprocessed signal (since it's usually near 1)
    norm_raw = raw_fgs1 / np.nanmean(raw_fgs1)
    norm_prep = prep_fgs1 / np.nanmean(prep_fgs1)

    fig, ax1 = plt.subplots(figsize=(10, 4))

    # Left axis: normalized flux comparison
    ax1.plot(norm_raw, label="Raw (normalized)", alpha=0.5, color="orange")
    ax1.plot(norm_prep, label="Preprocessed", alpha=0.8, color="cyan")
    ax1.set_xlabel("Time bins")
    ax1.set_ylabel("Normalized Flux")
    ax1.grid(alpha=0.3)
    ax1.legend()

    # Right axis: show raw counts scale for context
    ax2 = ax1.twinx()
    ax2.set_ylabel("Raw Counts", color="orange", alpha=0.6)
    ax2.tick_params(axis='y', labelcolor='orange', colors='orange')

    plt.title(f"FGS1 Light Curve Comparison — Planet {planet_id}")
    plt.tight_layout()
    plt.show()

# ---------------------------------------------------------
# EXAMPLE USAGE
# ---------------------------------------------------------

if __name__ == "__main__":
    # Example: view planet #0 in preprocessed mode
    visualize_planet(0, source="preprocessed")

    # Example: view planet #0 in raw mode (from parquet)
    visualize_planet(0, source="raw")

    compare_planet_signals(
        planet_idx=0
    )
    # plot_fgs1_comparison(raw_fgs1, prep_fgs1, planet_id)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

def compare_planet_signals(planet_idx, signal_v2_path='/kaggle/working/signal_v2.npy', raw_base_dir="./train"):
    """
    Compare raw and preprocessed signals for a given planet index.
    
    - Loads raw .parquet data from the ARIEL dataset folder
    - Loads preprocessed data from signal_v2.npy
    - Overlays FGS1 and AIRS-CH0 signals for visual comparison
    """

    # --- Load preprocessed data ---
    signal = np.load(signal_v2_path, allow_pickle=True)
    prep_fgs1, prep_airs = signal[planet_idx, 0], signal[planet_idx, 1]

    # --- Identify planet ID from folder structure ---
    planet_id = sorted(os.listdir(raw_base_dir))[planet_idx]
    print(f"🪐 Viewing Planet ID: {planet_id}")

    # --- Load raw data ---
    raw_fgs1_path = os.path.join(raw_base_dir, planet_id, "FGS1_signal_0.parquet")
    raw_airs_path = os.path.join(raw_base_dir, planet_id, "AIRS-CH0_signal_0.parquet")

    raw_fgs1 = pd.read_parquet(raw_fgs1_path).values
    raw_airs = pd.read_parquet(raw_airs_path).values

    # Flatten if needed
    if raw_fgs1.ndim > 1:
        raw_fgs1 = raw_fgs1.flatten()

    # --- Plot FGS1 (1D) ---
    plt.figure(figsize=(10, 4))
    plt.plot(raw_fgs1, label="Raw FGS1", alpha=0.6, color="orange")
    plt.plot(prep_fgs1, label="Preprocessed FGS1", alpha=0.8, color="cyan")
    plt.title(f"FGS1 Light Curve Comparison — Planet {planet_id}")
    plt.xlabel("Time bins")
    plt.ylabel("Flux")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

    # --- Plot AIRS-CH0 (2D heatmap and overlay) ---
    fig, ax = plt.subplots(1, 3, figsize=(15, 4))

    im0 = ax[0].imshow(raw_airs.T, aspect='auto', origin='lower', cmap='magma')
    ax[0].set_title("Raw AIRS-CH0")
    plt.colorbar(im0, ax=ax[0], fraction=0.046, pad=0.04)

    im1 = ax[1].imshow(prep_airs.T, aspect='auto', origin='lower', cmap='magma')
    ax[1].set_title("Preprocessed AIRS-CH0")
    plt.colorbar(im1, ax=ax[1], fraction=0.046, pad=0.04)

    mid_band = raw_airs.shape[1] // 2
    ax[2].plot(raw_airs[:, mid_band], label='Raw', alpha=0.6, color='orange')
    ax[2].plot(prep_airs[:, mid_band], label='Preprocessed', alpha=0.8, color='cyan')
    ax[2].set_title("Central Band Comparison")
    ax[2].set_xlabel("Time bin")
    ax[2].set_ylabel("Flux")
    ax[2].legend()
    ax[2].grid(alpha=0.3)

    fig.suptitle(f"AIRS-CH0 Comparison — Planet {planet_id}", fontsize=13)
    plt.tight_layout()
    plt.show()


In [ ]:
train_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2025/adc_info.csv')
train_star_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2025/train_star_info.csv')

train_labels = pd.read_csv('/kaggle/input/ariel-data-challenge-2025/train.csv',
                           index_col='planet_id')
wavelengths = pd.read_csv('/kaggle/input/ariel-data-challenge-2025/wavelengths.csv')
axis_info = pd.read_parquet('/kaggle/input/ariel-data-challenge-2025/axis_info.parquet')

In [ ]:
from scipy.signal import savgol_filter


MODEL_VERSION = "v2"
PRE_BINNED_TIME = 15

ROOT = "/kaggle/input/ariel-data-challenge-2025/"


# find transit zones
def phase_detector(signal_orig, smooth_window=11):
    signal = signal_orig.reshape(-1, 1).mean(-1)
    signal = savgol_filter(signal, smooth_window, 2)  # smooth
    first_derivative = np.gradient(signal)
    second_derivative = savgol_filter(np.gradient(savgol_filter(first_derivative, 41, 2)), 41, 2)

    local_min = (np.diff(np.sign(np.diff(second_derivative))) > 0).nonzero()[0] + 1
    local_max = (np.diff(np.sign(np.diff(second_derivative))) < 0).nonzero()[0] + 1
    
    if len(local_min) >= 2:
        top2_min_indices = local_min[np.argsort(second_derivative[local_min])[:2]]
    else:
        top2_min_indices = local_min
    
    if len(local_max) >= 2:
        top2_max_indices = local_max[np.argsort(second_derivative[local_max])[-2:]]
    else:
        top2_max_indices = local_max
    top2_min_indices.sort()
    top2_max_indices.sort()

    # 4 extrema of the 2nd derivative and 2 of the 1st
    phase1 = top2_min_indices[0]
    phase2 = top2_max_indices[0]
    phase3 = top2_max_indices[1]
    phase4 = top2_min_indices[1]
    phase5 = np.argmin(first_derivative)
    phase6 = np.argmax(first_derivative)

    return phase1, phase2, phase3, phase4, phase5, phase6


def get_breakpoints(x, smooth=19):
    bp1 = np.zeros(x.shape[0], dtype=np.int32)
    bp2 = np.zeros(x.shape[0], dtype=np.int32)
    bp3 = np.zeros(x.shape[0], dtype=np.int32)
    bp4 = np.zeros(x.shape[0], dtype=np.int32)
    bp5 = np.zeros(x.shape[0], dtype=np.int32)
    bp6 = np.zeros(x.shape[0], dtype=np.int32)
    for i in range(x.shape[0]):
        signal = x[i]
        p1, p2, p3, p4, p5, p6 = phase_detector(
            signal, smooth_window=smooth
        )
        bp1[i] = p1
        bp2[i] = p2
        bp3[i] = p3
        bp4[i] = p4
        bp5[i] = p5
        bp6[i] = p6
        
    return [bp1, bp2, bp3, bp4, bp5, bp6]

In [ ]:
%%writefile feature_engineering.py
from scipy.signal import savgol_filter
import warnings
from scipy.stats import kurtosis, skew
from scipy.signal import medfilt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from numpy.polynomial import Polynomial
from scipy.optimize import least_squares, minimize
from sklearn.metrics import mean_squared_error
from scipy.ndimage import median_filter


warnings.simplefilter("ignore")

A_BINNING = 15

# threshold values for identifying outliers
bad_low = 20
bad_up = 354

buf = 15  # for lower1, upper1
buf1 = 10  # for lower, mid1, mid2, upper


def calculate_weights(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    """
    Calculating weights for averaging based on SNR
    """
    max_len = a.shape[0] - 1
    if is_outlier:
        return np.ones_like(a[0]) / a.shape[-1]
    else:
        y_combined = np.concatenate([a[:max(lower - buf1, 1), :], a[min(upper + buf1, max_len):, :]], axis=0)
        ratio = y_combined.mean(0) / y_combined.std(0)
        return ratio / ratio.sum()


def calc_for_outliers(a, lower, upper):
    """
    Estimating transit depth for outlier cases
    """
    max_len = len(a) - 1
            
    if lower + buf < upper - buf:
        obs = a[lower + buf : upper - buf].mean()
    else:
        obs = a[lower : upper].mean()

    if lower - buf >= 10 and max_len - upper - buf >= 10:
        unobs = (np.median(a[:max(lower - buf, 1)]) + np.median(a[min(upper + buf, max_len):])) / 2
    elif lower >= max_len - upper:
        unobs = np.median(a[:max(lower - buf, 1)])
    else:
        unobs = np.median(a[min(upper + buf, max_len):])

    arr1 = 1 - (obs / unobs)
    arr2 = 1 - a[(lower + upper) // 2] / unobs

    return arr1, arr2


def calc_err(x_combined, y_combined, degree):
    """
    Calculating the error to obtain the optimal polynomial degree
    """
    max_len = 374 # hardcoded for BINNING=15
    
    poly_guess = np.polyfit(x_combined, y_combined, degree)
    inter = np.polyval(poly_guess, np.arange(max_len + 1))
    err = mean_squared_error(y_combined, inter[x_combined], squared=False)

    # penalizing rmse, high polynomial degree and small number of points in curve fitting.
    return err * degree**(1 - len(x_combined) / max_len)

    
def calc_depth_and_detrend(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False, unstable=False, fixed_degree=None):
    """
    Main function for transit depth estimation and detrending
    
    Parameters:
        - a: 1d numpy array of observation points
        - lower, mid1, mid2, upper, lower1, upper1: transit boundary points
        - is_outlier: boolean flag indicating if the data point is an outlier
        - unstable: if True, don't use curve fitting
        - fixed_degree: if not None, use provided degree; otherwise, find optimal degree
    
    Returns:
        - (arr1, arr2): tuple containing the averaged transit depth and the transit depth at mid-transit
    """
    max_len = len(a) - 1
    degree = 3
    
    if is_outlier:
        a /= a.mean()
        arr1, arr2 = calc_for_outliers(a, lower1, upper1)
    else:
        a /= a.mean()

        # region outside the transit
        x_combined = np.concatenate([np.arange(max(lower - buf1, 1)), np.arange(min(upper + buf1, max_len), max_len + 1)], axis=-1)
        y_combined = a[x_combined]
            
        if fixed_degree is None: # find the optimal degre
            best_val = 10**100
            for j in [1, 2, 3, 4, 5]:
                if calc_err(x_combined, y_combined, j) < best_val:
                    best_val = calc_err(x_combined, y_combined, j)
                    degree = j
        else:
            degree = fixed_degree

        obs = a[mid1 : mid2]
        
        if unstable: # no curve fitting
            unobs = y_combined.mean()
        else:
            poly_guess = np.polyfit(x_combined, y_combined, degree)         
            inter = np.polyval(poly_guess, np.arange(max_len + 1))
                
            a /= inter
            inter /= inter
            unobs = inter[mid1 : mid2]

        arr1 = 1 - np.mean(obs / unobs)
        arr2 = 1 - a[(lower1 + upper1) // 2] / unobs.mean()
                

    if np.isnan(arr1):
        arr1 = 0
    if np.isnan(arr2):
        arr2 = 0
        
    return arr1, arr2


def calc_slope(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    """
    Calculate transit wall steepness as slope between contact points
    """
    max_len = len(a) - 1
    
    if not (lower < mid1 < mid2 < upper) or is_outlier:
        return 0
    else:
        return ((a[mid1] - a[lower]) / (mid1 - lower) - (a[upper] - a[mid2]) / (upper - mid2)) / 2


def calc_slope_2(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    """
    Calculate transit bottom curvature as slope between mid-transit 
    and contact point
    """
    max_len = len(a) - 1
    
    if not (lower < mid1 < mid2 < upper) or is_outlier:
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        if not (mid1 < mid_ind < mid2):
            return 0
        return ((a[mid_ind] - a[mid1]) / (mid_ind - mid1) - (a[mid2] - a[mid_ind]) / (mid2 - mid_ind)) / 2


# calcluating slopes for unsimmetric cases
def calc_slope_2_left(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    max_len = len(a) - 1
    
    if not (lower < mid1 < mid2 < upper):
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        if not (mid1 < mid_ind < mid2):
            return 0
        return (a[mid_ind] - a[mid1]) / (mid_ind - mid1)


def calc_slope_2_right(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    max_len = len(a) - 1
    
    if not (lower < mid1 < mid2 < upper):
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        if not (mid1 < mid_ind < mid2):
            return 0
        return (a[mid2] - a[mid_ind]) / (mid2 - mid_ind)


# gradient slopes
def calc_curv_left(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    if not (lower < mid1 < mid2 < upper) or is_outlier:
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        a = savgol_filter(np.gradient(a), 41, 1)
        return (a[mid_ind] - a[mid1]) / (mid_ind - mid1)


def calc_curv_right(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    if not (lower < mid1 < mid2 < upper) or is_outlier:
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        a = savgol_filter(np.gradient(a), 41, 1)
        return (a[mid2] - a[mid_ind]) / (mid2 - mid_ind)


def calc_perc(a, lower, upper, q, is_outlier=False):
    """
    Calculate the percentile of the transit depth, assumes the input signal is already detrended
    """
    if is_outlier:
        return 0
    return np.quantile(1 - a[lower : upper], q)
    

def feature_engineering(star_info, data):
    """
    Prepares features for training or inference
    
    Parameters:
        - star_info: star metadata DataFrame
        - data: 3d data array (samples, time, frequencies)
    
    Returns:
        tuple of DataFrame and outliers mask
    """
    df = pd.DataFrame()

    cut_inf, cut_sup = 36, 318
    
    signal = np.concatenate(
        [data[:, :, 0][:, :, None], data[:, :, cut_inf:cut_sup]], axis=2
    )
    max_len = signal.shape[1] - 1
        
    lower, mid1, mid2, upper, lower1, upper1 = get_breakpoints(signal[:, :, 1:].mean(-1))
    boundaries = (lower, mid1, mid2, upper, lower1, upper1) 

    # identifying outliers
    outliers = (np.array(lower1) < bad_low) | (np.array(upper1) > bad_up) | (np.array(lower) < bad_low) | (np.array(upper) > bad_up)
    for i in range(signal.shape[0]):
        if not (lower[i] < mid1[i] < mid2[i] < upper[i]):
            outliers[i] = 1
    
    signal_mean_raw = np.zeros(signal.shape[:2])
    for i in tqdm(range(signal_mean_raw.shape[0])): # weighted averaging along frequency dimension
        weights = calculate_weights(signal[i, :, 1:], *(b[i] for b in boundaries), outliers[i])
        signal_mean_raw[i, :] = (signal[i, :, 1:] @ weights.T).T

    signal_mean = savgol_filter(signal_mean_raw, 11, 1)

    # frequency set for precise depth estimation via curve fitting (less robust)
    good_waves = [1, 6, 11, 16, 21, 26, 31, 36, 41, 51, 61, 71, 76, 81, 86, 91, 96, 101, 106, 111, 121, 131, 141, 151, 161, 171, 196, 201, 206]
    for i in tqdm(range(len(signal_mean))):

        # filter very bad cases :) (exclude from training, use larger sigma for prediction)
        df.loc[i, 'very_bad'] = (lower1[i] < bad_low // 2 or upper1[i] > max_len - (max_len - bad_up) // 2)
        if os.environ["PREPROCESS_MODE"] == 'train' and star_info.loc[i, 'planet_id'] in [2486733311, 2554492145]:
            df.loc[i, 'very_bad'] = True


        # averaged and mid-transit depth estimation
        fake_avg, fake_mid = calc_depth_and_detrend(signal_mean[i].copy(), *(b[i] for b in boundaries), outliers[i], unstable=True)
        fake_avg_2, fake_mid_2 = calc_depth_and_detrend(signal_mean[i].copy(), *(b[i] for b in boundaries), outliers[i], fixed_degree=3)
        df.loc[i, 'average_depth'], df.loc[i, 'mid_depth'] = calc_depth_and_detrend(signal_mean[i], *(b[i] for b in boundaries), outliers[i])

        norm_coef = (1 - df.loc[i, 'average_depth']) / (1 - fake_avg)
        norm_coef_mid = (1 - df.loc[i, 'mid_depth']) / (1 - fake_mid)
        norm_coef_2 = (1 - df.loc[i, 'average_depth']) / (1 - fake_avg_2)
                        
        fg1_signal = signal[i, :, 0].copy()
        fg1_signal = savgol_filter(fg1_signal, 11, 1)
        fg1_slope = savgol_filter(signal[i, :, 0].copy(), 41, 2)
        _, _ = calc_depth_and_detrend(fg1_slope, *(b[i] for b in boundaries), outliers[i], fixed_degree=3) # for detrending
        
        df.loc[i, 'fg1_average_depth'], df.loc[i, 'fg1_mid_depth'] = calc_depth_and_detrend(fg1_signal, *(b[i] for b in boundaries), 
                                                                                            outliers[i], 
                                                                                            fixed_degree=3)

        
        # transit depth percentiles
        for q in [0.01, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
            df.loc[i, f'q_1_{q}'] = calc_perc(signal_mean[i], mid1[i], mid2[i], q, outliers[i])
            df.loc[i, f'q_2_{q}'] = calc_perc(signal_mean[i], lower[i] - buf1, upper[i] + buf1, q, outliers[i])
            df.loc[i, f'q_3_{q}'] = calc_perc(signal_mean[i], lower[i], upper[i], q, outliers[i]) 
        for q in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9]:
            df.loc[i, f'fg1_q_1_{q}'] = calc_perc(fg1_signal, mid1[i], mid2[i], q, outliers[i])
            df.loc[i, f'fg1_q_2_{q}'] = calc_perc(fg1_signal, lower[i], upper[i], q, outliers[i])
            df.loc[i, f'fg1_q_3_{q}'] = calc_perc(fg1_signal, lower[i] - buf1, upper[i] + buf1, q, outliers[i])

        
        # slope features
        df.loc[i, 'slope'] = calc_slope(signal_mean[i], *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'slope_2'] = calc_slope_2(signal_mean[i], *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'slope_2_left'] = calc_slope_2_left(signal_mean[i], *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'slope_2_right'] = calc_slope_2_right(signal_mean[i], *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'slope_g'] = max(0, -df.loc[i, 'slope_2'])**0.5
                          
        df.loc[i, 'fg1_slope'] = calc_slope(fg1_slope, *(b[i] for b in boundaries), outliers[i])     
        df.loc[i, 'fg1_slope_2'] = calc_slope_2(fg1_slope, *(b[i] for b in boundaries), outliers[i])      
        df.loc[i, 'fg1_slope_g'] = max(0, -df.loc[i, 'fg1_slope_2'])**0.5
        df.loc[i, 'fg1_curv_left'] = calc_curv_left(fg1_slope, *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'fg1_curv_right'] = calc_curv_right(fg1_slope, *(b[i] for b in boundaries), outliers[i])

        
        # combinations with slopes
        df.loc[i, 'slope_rel'] = df.loc[i, 'slope_2'] * df.loc[i, 'average_depth']
        df.loc[i, 'fg1_slope_T'] = df.loc[i, 'fg1_slope_2'] * star_info.loc[i, 'Ts'] 
        df.loc[i, 'fg1_slope_rel'] = df.loc[i, 'fg1_slope_2'] * df.loc[i, 'fg1_average_depth']
        df.loc[i, 'fg1_slope_g_rel'] = df.loc[i, 'fg1_slope_g'] * df.loc[i, 'fg1_average_depth']
        
        for q in [0.01, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
            df.loc[i, f'slope_q_{q}'] = df.loc[i, 'slope_2'] * df.loc[i, f'q_1_{q}']
            df.loc[i, f'slope_q_{q}_2'] = df.loc[i, 'slope_2'] * df.loc[i, f'q_2_{q}']

        
        # other features
        df.loc[i, 't14'] = upper[i] - lower[i]
        df.loc[i, 't23'] = mid2[i] - mid1[i]
        df.loc[i, 'time'] = (mid1[i] - lower[i]) / (upper[i] - lower[i])        
        df.loc[i, 'P_mul_Rs'] = star_info.loc[i, 'P'] * star_info.loc[i, 'Rs']
        df.loc[i, 'P_div_Rs'] = star_info.loc[i, 'P'] / star_info.loc[i, 'Rs']
        
        step = 5
        max_rel = 0
        min_rel = 1
        meaning = 60 # window size for frequency averagning
        
        for j in range(1, signal.shape[-1] - meaning + 1, step):
            if j <= 80:
                meaning = 20
            elif j <= 180:
                meaning = 30
            else:
                meaning = 60

            cur_mean = signal[i, :, j : min(j + meaning, signal.shape[-1])].mean(-1)

            # median filter
            if not outliers[i]:
                if j >= 180:
                    med_kernel = 31
                else:
                    med_kernel = 21
                cur_mean = median_filter(cur_mean, size=med_kernel, mode="constant")
                           
            cur_mean = savgol_filter(cur_mean, 11, 1)
            
            df.loc[i, f'averaged_{j}_unstable'], df.loc[i, f'mid_{j}_unstable'] = calc_depth_and_detrend(cur_mean.copy(), *(b[i] for b in boundaries), 
                                                                                                            outliers[i],
                                                                                                            unstable=True)  
            if not outliers[i]:
                df.loc[i, f'averaged_{j}_unstable'] = 1 - (1 - df.loc[i, f'averaged_{j}_unstable']) * norm_coef
  
            if j in good_waves:
                df.loc[i, f'averaged_{j}'], df.loc[i, f'mid_{j}'] = calc_depth_and_detrend(cur_mean, *(b[i] for b in boundaries),
                                                                                              outliers[i],
                                                                                              fixed_degree=3)
                if not outliers[i]:
                    df.loc[i, f'averaged_{j}'] = 1 - (1 - df.loc[i, f'averaged_{j}']) * norm_coef_2

            
            # percentiles
            for q in [0.1, 0.15, 0.2]:
                if j in good_waves and not outliers[i]:
                    df.loc[i, f'q_w_{j}_{q}'] = calc_perc(cur_mean, mid1[i], mid2[i], q, outliers[i])
                elif outliers[i] and mid1[i] < mid2[i]:
                    x_combined = np.concatenate([np.arange(max(lower[i] - buf1, 1)), np.arange(min(upper[i] + buf1, max_len), max_len + 1)], axis=-1)
                    mid_q = np.quantile(cur_mean[mid1[i] : mid2[i]], q)
                    df.loc[i, f'q_w_{j}_{q}'] = 1 - mid_q / cur_mean[x_combined].mean()
                else:
                    df.loc[i, f'q_w_{j}_{q}'] = 0
           
            
            max_rel = max(max_rel, df.loc[i, f'averaged_{j}_unstable'])
            min_rel = min(min_rel, df.loc[i, f'averaged_{j}_unstable'])

  
            # slope combinations
            df.loc[i, f'averaged_slope_{j}'] = df.loc[i, f'averaged_{j}_unstable'] * df.loc[i, 'slope_2']
            df.loc[i, f'averaged_slope_g_{j}'] = df.loc[i, f'averaged_{j}_unstable'] * df.loc[i, 'slope_g']

        
        # large amplitude     
        if max_rel - min_rel >= 0.005:
            df.loc[i, 'very_bad'] = True

    
    df['Rs'] = star_info['Rs']
    df['Ms'] = star_info['Ms']
    df['Ts'] = star_info['Ts']
    df['sma'] = star_info['sma']  
    df['g'] = np.log10(star_info['Ms'] / (star_info['Rs']**2))
    df['g_T'] = df['g'] * star_info['Ts']
    df['big_rs'] = (star_info['Rs'] > np.quantile(star_info['Rs'].values, 0.97))
    
    df['outliers'] = outliers

    df = df.fillna(0)
    
    return outliers, df

In [ ]:
exec(open('feature_engineering.py', 'r').read())
outliers, train = feature_engineering(train_star_info, data_train)
outliers = np.arange(train.shape[0])[outliers]
len(outliers)

In [ ]:
# feature engineering file 2
# %%writefile feature_engineering.py

from scipy.signal import savgol_filter
import warnings
from scipy.stats import kurtosis, skew
from scipy.signal import medfilt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from numpy.polynomial import Polynomial
from scipy.optimize import least_squares, minimize
from sklearn.metrics import mean_squared_error
from scipy.ndimage import median_filter
import numpy as np
import pandas as pd
from tqdm import tqdm
import os


warnings.simplefilter("ignore")

A_BINNING = 15

# threshold values for identifying outliers
bad_low = 20
bad_up = 354

buf = 15  # for lower1, upper1
buf1 = 20  # for lower, mid1, mid2, upper


class FeatureEngineeringConfig:
    """
    Configuration class to store dataset-level statistics
    calculated from training data only
    """
    def __init__(self):
        self.rs_threshold_97 = None
    
    def fit(self, train_star_info):
        """Calculate statistics from training data"""
        self.rs_threshold_97 = np.quantile(train_star_info['Rs'].values, 0.97)
        return self
    
    def save(self, filepath):
        """Save config for later use"""
        import pickle
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)
    
    @staticmethod
    def load(filepath):
        """Load saved config"""
        import pickle
        with open(filepath, 'rb') as f:
            return pickle.load(f)


def calculate_weights(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    """
    Calculating weights for averaging based on SNR
    """
    max_len = a.shape[0] - 1
    if is_outlier:
        return np.ones_like(a[0]) / a.shape[-1]
    else:
        y_combined = np.concatenate([a[:max(lower - buf1, 1), :], a[min(upper + buf1, max_len):, :]], axis=0)
        ratio = y_combined.mean(0) / y_combined.std(0)
        return ratio / ratio.sum()


def calc_for_outliers(a, lower, upper):
    """
    Estimating transit depth for outlier cases
    """
    max_len = len(a) - 1
            
    if lower + buf < upper - buf:
        obs = a[lower + buf : upper - buf].mean()
    else:
        obs = a[lower : upper].mean()

    if lower - buf >= 10 and max_len - upper - buf >= 10:
        unobs = (np.median(a[:max(lower - buf, 1)]) + np.median(a[min(upper + buf, max_len):])) / 2
    elif lower >= max_len - upper:
        unobs = np.median(a[:max(lower - buf, 1)])
    else:
        unobs = np.median(a[min(upper + buf, max_len):])

    arr1 = 1 - (obs / unobs)
    arr2 = 1 - a[(lower + upper) // 2] / unobs

    return arr1, arr2


def calc_err(x_combined, y_combined, degree):
    """
    Calculating the error to obtain the optimal polynomial degree
    """
    max_len = 374 # hardcoded for BINNING=15
    
    poly_guess = np.polyfit(x_combined, y_combined, degree)
    inter = np.polyval(poly_guess, np.arange(max_len + 1))
    err = mean_squared_error(y_combined, inter[x_combined], squared=False)

    # penalizing rmse, high polynomial degree and small number of points in curve fitting.
    return err * degree**(1 - len(x_combined) / max_len)

    
def calc_depth_and_detrend(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False, unstable=False, fixed_degree=2):
    """
    Main function for transit depth estimation and detrending
    
    Parameters:
        - a: 1d numpy array of observation points
        - lower, mid1, mid2, upper, lower1, upper1: transit boundary points
        - is_outlier: boolean flag indicating if the data point is an outlier
        - unstable: if True, don't use curve fitting
        - fixed_degree: if not None, use provided degree; otherwise, find optimal degree
    
    Returns:
        - (arr1, arr2): tuple containing the averaged transit depth and the transit depth at mid-transit
    """
    max_len = len(a) - 1
    degree = 2
    
    if is_outlier:
        a /= a.mean()
        arr1, arr2 = calc_for_outliers(a, lower1, upper1)
    else:
        a /= a.mean()

        # region outside the transit
        x_combined = np.concatenate([np.arange(max(lower - buf1, 1)), np.arange(min(upper + buf1, max_len), max_len + 1)], axis=-1)
        y_combined = a[x_combined]
            
        if fixed_degree is None: # find the optimal degree
            best_val = 10**100
            for j in [1, 2, 3, 4, 5]:
                if calc_err(x_combined, y_combined, j) < best_val:
                    best_val = calc_err(x_combined, y_combined, j)
                    degree = j
        else:
            degree = fixed_degree

        obs = a[mid1 : mid2]
        
        if unstable: # no curve fitting
            unobs = y_combined.mean()
        else:
            poly_guess = np.polyfit(x_combined, y_combined, degree)         
            inter = np.polyval(poly_guess, np.arange(max_len + 1))
                
            a /= inter
            inter /= inter
            unobs = inter[mid1 : mid2]

        arr1 = 1 - np.mean(obs / unobs)
        arr2 = 1 - a[(lower1 + upper1) // 2] / unobs.mean()
                

    if np.isnan(arr1):
        arr1 = 0
    if np.isnan(arr2):
        arr2 = 0
        
    return arr1, arr2


def calc_slope(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    """
    Calculate transit wall steepness as slope between contact points
    """
    max_len = len(a) - 1
    
    if not (lower < mid1 < mid2 < upper) or is_outlier:
        return 0
    else:
        return ((a[mid1] - a[lower]) / (mid1 - lower) - (a[upper] - a[mid2]) / (upper - mid2)) / 2


def calc_slope_2(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    """
    Calculate transit bottom curvature as slope between mid-transit 
    and contact point
    """
    max_len = len(a) - 1
    
    if not (lower < mid1 < mid2 < upper) or is_outlier:
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        if not (mid1 < mid_ind < mid2):
            return 0
        return ((a[mid_ind] - a[mid1]) / (mid_ind - mid1) - (a[mid2] - a[mid_ind]) / (mid2 - mid_ind)) / 2


# calculating slopes for asymmetric cases
def calc_slope_2_left(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    max_len = len(a) - 1
    
    if not (lower < mid1 < mid2 < upper):
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        if not (mid1 < mid_ind < mid2):
            return 0
        return (a[mid_ind] - a[mid1]) / (mid_ind - mid1)


def calc_slope_2_right(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    max_len = len(a) - 1
    
    if not (lower < mid1 < mid2 < upper):
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        if not (mid1 < mid_ind < mid2):
            return 0
        return (a[mid2] - a[mid_ind]) / (mid2 - mid_ind)


# gradient slopes
def calc_curv_left(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    if not (lower < mid1 < mid2 < upper) or is_outlier:
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        a = savgol_filter(np.gradient(a), 41, 1)
        return (a[mid_ind] - a[mid1]) / (mid_ind - mid1)


def calc_curv_right(a, lower, mid1, mid2, upper, lower1, upper1, is_outlier=False):
    if not (lower < mid1 < mid2 < upper) or is_outlier:
        return 0
    else:
        mid_ind = (mid1 + mid2) // 2
        a = savgol_filter(np.gradient(a), 41, 1)
        return (a[mid2] - a[mid_ind]) / (mid2 - mid_ind)


def calc_perc(a, lower, upper, q, is_outlier=False):
    """
    Calculate the percentile of the transit depth, assumes the input signal is already detrended
    """
    if is_outlier:
        return 0
    return np.quantile(1 - a[lower : upper], q)
    

def feature_engineering(star_info, data, config=None, is_training=False):
    """
    Prepares features for training or inference
    
    Parameters:
        - star_info: star metadata DataFrame
        - data: 3d data array (samples, time, frequencies)
        - config: FeatureEngineeringConfig object (required if not training)
        - is_training: whether this is training data (will fit config if True)
    
    Returns:
        tuple of (outliers mask, DataFrame, config)
    """
    df = pd.DataFrame()

    # Initialize or validate config
    if is_training:
        if config is None:
            config = FeatureEngineeringConfig()
        config.fit(star_info)
    else:
        if config is None:
            raise ValueError("config must be provided when is_training=False")

    cut_inf, cut_sup = 36, 318
    
    signal = np.concatenate(
        [data[:, :, 0][:, :, None], data[:, :, cut_inf:cut_sup]], axis=2
    )
    max_len = signal.shape[1] - 1
        
    lower, mid1, mid2, upper, lower1, upper1 = get_breakpoints(signal[:, :, 1:].mean(-1))
    boundaries = (lower, mid1, mid2, upper, lower1, upper1) 

    # identifying outliers
    outliers = (np.array(lower1) < bad_low) | (np.array(upper1) > bad_up) | (np.array(lower) < bad_low) | (np.array(upper) > bad_up)
    for i in range(signal.shape[0]):
        if not (lower[i] < mid1[i] < mid2[i] < upper[i]):
            outliers[i] = 1
    
    signal_mean_raw = np.zeros(signal.shape[:2])
    for i in tqdm(range(signal_mean_raw.shape[0])): # weighted averaging along frequency dimension
        weights = calculate_weights(signal[i, :, 1:], *(b[i] for b in boundaries), outliers[i])
        signal_mean_raw[i, :] = (signal[i, :, 1:] @ weights.T).T

    signal_mean = savgol_filter(signal_mean_raw, 11, 1)

    # frequency set for precise depth estimation via curve fitting (less robust)
    good_waves = [1, 6, 11, 16, 21, 26, 31, 36, 41, 51, 61, 71, 76, 81, 86, 91, 96, 101, 106, 111, 121, 131, 141, 151, 161, 171, 196, 201, 206]
    
    for i in tqdm(range(len(signal_mean))):

        # Filter very bad cases based on data characteristics only
        df.loc[i, 'very_bad'] = (lower1[i] < bad_low // 2 or upper1[i] > max_len - (max_len - bad_up) // 2)

        # averaged and mid-transit depth estimation
        fake_avg, fake_mid = calc_depth_and_detrend(signal_mean[i].copy(), *(b[i] for b in boundaries), outliers[i], unstable=True)
        fake_avg_2, fake_mid_2 = calc_depth_and_detrend(signal_mean[i].copy(), *(b[i] for b in boundaries), outliers[i], fixed_degree=3)
        df.loc[i, 'average_depth'], df.loc[i, 'mid_depth'] = calc_depth_and_detrend(signal_mean[i], *(b[i] for b in boundaries), outliers[i])

        norm_coef = (1 - df.loc[i, 'average_depth']) / (1 - fake_avg)
        norm_coef_mid = (1 - df.loc[i, 'mid_depth']) / (1 - fake_mid)
        norm_coef_2 = (1 - df.loc[i, 'average_depth']) / (1 - fake_avg_2)
                        
        fg1_signal = signal[i, :, 0].copy()
        fg1_signal = savgol_filter(fg1_signal, 11, 1)
        fg1_slope = savgol_filter(signal[i, :, 0].copy(), 41, 2)
        _, _ = calc_depth_and_detrend(fg1_slope, *(b[i] for b in boundaries), outliers[i], fixed_degree=3) # for detrending
        
        df.loc[i, 'fg1_average_depth'], df.loc[i, 'fg1_mid_depth'] = calc_depth_and_detrend(fg1_signal, *(b[i] for b in boundaries), 
                                                                                            outliers[i], 
                                                                                            fixed_degree=3)

        
        # transit depth percentiles
        for q in [0.01, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
            df.loc[i, f'q_1_{q}'] = calc_perc(signal_mean[i], mid1[i], mid2[i], q, outliers[i])
            df.loc[i, f'q_2_{q}'] = calc_perc(signal_mean[i], lower[i] - buf1, upper[i] + buf1, q, outliers[i])
            df.loc[i, f'q_3_{q}'] = calc_perc(signal_mean[i], lower[i], upper[i], q, outliers[i]) 
        for q in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9]:
            df.loc[i, f'fg1_q_1_{q}'] = calc_perc(fg1_signal, mid1[i], mid2[i], q, outliers[i])
            df.loc[i, f'fg1_q_2_{q}'] = calc_perc(fg1_signal, lower[i], upper[i], q, outliers[i])
            df.loc[i, f'fg1_q_3_{q}'] = calc_perc(fg1_signal, lower[i] - buf1, upper[i] + buf1, q, outliers[i])

        
        # slope features
        df.loc[i, 'slope'] = calc_slope(signal_mean[i], *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'slope_2'] = calc_slope_2(signal_mean[i], *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'slope_2_left'] = calc_slope_2_left(signal_mean[i], *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'slope_2_right'] = calc_slope_2_right(signal_mean[i], *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'slope_g'] = max(0, -df.loc[i, 'slope_2'])**0.5
                          
        df.loc[i, 'fg1_slope'] = calc_slope(fg1_slope, *(b[i] for b in boundaries), outliers[i])     
        df.loc[i, 'fg1_slope_2'] = calc_slope_2(fg1_slope, *(b[i] for b in boundaries), outliers[i])      
        df.loc[i, 'fg1_slope_g'] = max(0, -df.loc[i, 'fg1_slope_2'])**0.5
        df.loc[i, 'fg1_curv_left'] = calc_curv_left(fg1_slope, *(b[i] for b in boundaries), outliers[i])
        df.loc[i, 'fg1_curv_right'] = calc_curv_right(fg1_slope, *(b[i] for b in boundaries), outliers[i])

        
        # combinations with slopes
        df.loc[i, 'slope_rel'] = df.loc[i, 'slope_2'] * df.loc[i, 'average_depth']
        df.loc[i, 'fg1_slope_T'] = df.loc[i, 'fg1_slope_2'] * star_info.loc[i, 'Ts'] 
        df.loc[i, 'fg1_slope_rel'] = df.loc[i, 'fg1_slope_2'] * df.loc[i, 'fg1_average_depth']
        df.loc[i, 'fg1_slope_g_rel'] = df.loc[i, 'fg1_slope_g'] * df.loc[i, 'fg1_average_depth']
        
        for q in [0.01, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
            df.loc[i, f'slope_q_{q}'] = df.loc[i, 'slope_2'] * df.loc[i, f'q_1_{q}']
            df.loc[i, f'slope_q_{q}_2'] = df.loc[i, 'slope_2'] * df.loc[i, f'q_2_{q}']

        
        # other features
        df.loc[i, 't14'] = upper[i] - lower[i]
        df.loc[i, 't23'] = mid2[i] - mid1[i]
        df.loc[i, 'time'] = (mid1[i] - lower[i]) / (upper[i] - lower[i])        
        df.loc[i, 'P_mul_Rs'] = star_info.loc[i, 'P'] * star_info.loc[i, 'Rs']
        df.loc[i, 'P_div_Rs'] = star_info.loc[i, 'P'] / star_info.loc[i, 'Rs']
        
        step = 5
        max_rel = 0
        min_rel = 1
        meaning = 60 # window size for frequency averaging
        
        for j in range(1, signal.shape[-1] - meaning + 1, step):
            if j <= 80:
                meaning = 20
            elif j <= 180:
                meaning = 30
            else:
                meaning = 60

            cur_mean = signal[i, :, j : min(j + meaning, signal.shape[-1])].mean(-1)

            # median filter
            if not outliers[i]:
                if j >= 180:
                    med_kernel = 31
                else:
                    med_kernel = 21
                cur_mean = median_filter(cur_mean, size=med_kernel, mode="constant")
                           
            cur_mean = savgol_filter(cur_mean, 11, 1)
            
            df.loc[i, f'averaged_{j}_unstable'], df.loc[i, f'mid_{j}_unstable'] = calc_depth_and_detrend(cur_mean.copy(), *(b[i] for b in boundaries), 
                                                                                                            outliers[i],
                                                                                                            unstable=True)  
            if not outliers[i]:
                df.loc[i, f'averaged_{j}_unstable'] = 1 - (1 - df.loc[i, f'averaged_{j}_unstable']) * norm_coef
  
            if j in good_waves:
                df.loc[i, f'averaged_{j}'], df.loc[i, f'mid_{j}'] = calc_depth_and_detrend(cur_mean, *(b[i] for b in boundaries),
                                                                                              outliers[i],
                                                                                              fixed_degree=3)
                if not outliers[i]:
                    df.loc[i, f'averaged_{j}'] = 1 - (1 - df.loc[i, f'averaged_{j}']) * norm_coef_2

            
            # percentiles
            for q in [0.1, 0.15, 0.2]:
                if j in good_waves and not outliers[i]:
                    df.loc[i, f'q_w_{j}_{q}'] = calc_perc(cur_mean, mid1[i], mid2[i], q, outliers[i])
                elif outliers[i] and mid1[i] < mid2[i]:
                    x_combined = np.concatenate([np.arange(max(lower[i] - buf1, 1)), np.arange(min(upper[i] + buf1, max_len), max_len + 1)], axis=-1)
                    mid_q = np.quantile(cur_mean[mid1[i] : mid2[i]], q)
                    df.loc[i, f'q_w_{j}_{q}'] = 1 - mid_q / cur_mean[x_combined].mean()
                else:
                    df.loc[i, f'q_w_{j}_{q}'] = 0
           
            
            max_rel = max(max_rel, df.loc[i, f'averaged_{j}_unstable'])
            min_rel = min(min_rel, df.loc[i, f'averaged_{j}_unstable'])

  
            # slope combinations
            df.loc[i, f'averaged_slope_{j}'] = df.loc[i, f'averaged_{j}_unstable'] * df.loc[i, 'slope_2']
            df.loc[i, f'averaged_slope_g_{j}'] = df.loc[i, f'averaged_{j}_unstable'] * df.loc[i, 'slope_g']

        
        # large amplitude     
        if max_rel - min_rel >= 0.005:
            df.loc[i, 'very_bad'] = True

    
    df['Rs'] = star_info['Rs']
    df['Ms'] = star_info['Ms']
    df['Ts'] = star_info['Ts']
    df['sma'] = star_info['sma']  
    df['g'] = np.log10(star_info['Ms'] / (star_info['Rs']**2))
    df['g_T'] = df['g'] * star_info['Ts']
    
    # Use pre-calculated threshold from training data
    df['big_rs'] = (star_info['Rs'] > config.rs_threshold_97)
    
    df['outliers'] = outliers

    df = df.fillna(0)
    
    return outliers, df, config

In [ ]:
# Training
outliers_train, df_train, config = feature_engineering(
    train_star_info, 
    data_train, 
    is_training=True
)

# Save config for later
config.save('feature_config.pkl')

# # Inference/Testing
# config = FeatureEngineeringConfig.load('feature_config.pkl')
# outliers_test, df_test, _ = feature_engineering(
#     test_star_info, 
#     test_data, 
#     config=config,
#     is_training=False
# )

In [ ]:
# Inference/Testing

# test_star_info = f"{DATA_ROOT}/test_star_info.csv"
# config = FeatureEngineeringConfig.load('feature_config.pkl')
# outliers_test, df_test, _ = feature_engineering(
#     test_star_info, 
#     test_data, 
#     config=config,
#     is_training=False
# )

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.utils.validation import check_is_fitted


class CustomRidge(BaseEstimator):
    """
    Provides two models: main model for normal cases and outlier-specific model
    """
    def __init__(self):
        self.main = Ridge(alpha=3e-2) 
        self.outliers = Ridge(alpha=3e-1)

        self.main_scaler = RobustScaler()
        self.outliers_scaler = RobustScaler()


    def fit(self, X, y):
        groups = X['outliers']
        X = X.drop(columns=['outliers'])

        main_mask = (groups == 0).values
        main_mask[X.loc[:, 'very_bad'] == True] = 0
        
        X_main = self.main_scaler.fit_transform(X[main_mask])

        self.main.fit(self.main_scaler.transform(X[main_mask]), y[main_mask])
        self.outliers.fit(self.outliers_scaler.fit_transform(X), y)     
        self.pred_shape = y.shape[-1]

        return self

    def predict(self, X):
        groups = X['outliers']
        X = X.drop(columns=['outliers'])
        
        predictions = np.zeros((X.shape[0], self.pred_shape))

        main_mask = (groups == 0).values
        if main_mask.sum():
            predictions[main_mask] = self.main.predict(self.main_scaler.transform(X[main_mask]))                                                 
        if (~main_mask).sum():
            predictions[~main_mask] = self.outliers.predict(self.outliers_scaler.transform(X[~main_mask]))
        
        return predictions


model = CustomRidge()

oof_pred = cross_val_predict(model, train, train_labels.values, cv=100)

print(f"# R2 score: {r2_score(train_labels, oof_pred):.3f}")

sigma_pred = mean_squared_error(train_labels, oof_pred, squared=False)
  
print(f"# Root mean squared error: {sigma_pred:.6f}")

col = 1
plt.scatter(oof_pred[:,col], train_labels.iloc[:,col], s=15, c='lightgreen')
plt.gca().set_aspect('equal')
plt.xlabel('y_pred')
plt.ylabel('y_true')
plt.title('Comparing y_true and y_pred')
plt.show()

# clipping
oof_pred = np.maximum(oof_pred, 0.003)
oof_pred = np.minimum(oof_pred, 0.1)

In [ ]:
# # Split manually before CV
# from sklearn.model_selection import train_test_split

# X_train, X_valid, y_train, y_valid = train_test_split(
#     train, train_labels, test_size=0.2, random_state=42
# )

# model.fit(X_train, y_train)
# y_pred = model.predict(X_valid)

# print("R2:", r2_score(y_valid, y_pred))
# print("RMSE:", mean_squared_error(y_valid, y_pred, squared=False))


In [ ]:
# from sklearn.model_selection import KFold, cross_val_score

# cv = KFold(n_splits=5, shuffle=True, random_state=42)
# scores = cross_val_score(model, train, train_labels.values, cv=cv, scoring='r2')
# print("Mean R²:", np.mean(scores), "±", np.std(scores))



In [ ]:
# print(train.shape, train_labels.shape)
# print("Same index:", (train.index == train_labels.index).all())
# train, train_labels = train.align(train_labels, axis=0, join='inner')
# print("Same index:", (train.index == train_labels.index).all())


In [ ]:
# from sklearn.utils import shuffle

# y_shuffled = shuffle(train_labels)
# shuffled_score = np.mean(cross_val_score(model, train, y_shuffled, cv=5, scoring='r2'))
# print("Shuffled-label R²:", shuffled_score)


In [ ]:
# from sklearn.model_selection import learning_curve

# train_sizes, train_scores, test_scores = learning_curve(
#     model, train, train_labels.values, cv=5, scoring='r2'
# )


In [ ]:
# print(train_scores, test_scores)

In [ ]:
class SigmaPredictor:
    """
    Class for sigma predicting
    """
    def __init__(self):
        self.sigmas = {}
        
    def fit(self, y_pred, y_true, outliers, very_bad):        
        outliers = [i for i in outliers if i not in very_bad]

        self.sigmas['outliers'] = self._calc(y_pred[outliers], y_true[outliers]) * 5

        main = self._del_outliers(np.ones(len(y_pred), dtype=bool), outliers + list(very_bad))
        self.sigmas['main'] = self._calc(y_pred[main], y_true[main])

        print({ k: v.mean() for k, v in self.sigmas.items() })

    def predict(self, sigma_pred, y_pred, outliers, very_bad, bootstrap_preds=None):
        if len(outliers) > 0:
            sigma_pred[outliers] = self.sigmas['outliers']

        main = self._del_outliers(np.ones(len(y_pred), dtype=bool), outliers)
        if main.sum() > 0:
            sigma_pred[main] = self.sigmas['main']

        W1 = 0.75
        W2 = 1.0 - W1
        
        sigma_pred[main, :] = bootstrap_preds[main, :] * W1 + sigma_pred[main, :] * W2
        sigma_pred[outliers] = bootstrap_preds[outliers] * 1.5

        sigma_pred[very_bad] = 0.003 
        for i in very_bad:
            if i in outliers:
                continue
            sigma_pred[i, :] = 0.5 * bootstrap_preds[i, :] + 0.5 * sigma_pred[i, :]

        return sigma_pred
        

    def _calc(self, y_pred, y_true): # calculate rmse for each frequency
        sigmas = []
        for i in range(y_pred.shape[1]):
            sigmas.append(mean_squared_error(y_pred[:, i], y_true[:, i], squared=False))
        return np.array(sigmas)

    def _del_outliers(self, mask, outliers):
        for i in range(len(mask)):
            if i in outliers:
                mask[i] = False
        return mask                         


def postprocessing(pred_array, index, sigma_pred, sigma_predictor, outliers, very_bad, bootstrap_preds=None, column_names=None):
    """
    Creates a submission DataFrame with mean predictions and uncertainties.

    Parameters:
    - pred_array: ndarray of shape (n_samples, 283)
    - index: pandas.Index of length n_samples
    - sigma_pred: float or ndarray of shape (n_samples, 283)
    - column_names: list of wavelength column names (optional)

    Returns:
    - df: DataFrame of shape (n_samples, 566)
    """
    n_samples, n_waves = pred_array.shape

    if column_names is None:
        column_names = [f"wl_{i+1}" for i in range(n_waves)]
    
    sigma_pred = sigma_predictor.predict(np.zeros_like(pred_array), pred_array, outliers, very_bad, bootstrap_preds=bootstrap_preds)

    # Safety check
    assert sigma_pred.shape == pred_array.shape, "sigma_pred must match shape of pred_array"
    assert len(index) == n_samples, "Index length must match number of rows"

    df_mean = pd.DataFrame(pred_array.clip(0, None), index=index, columns=column_names)
    df_sigma = pd.DataFrame(sigma_pred, index=index, columns=[f"sigma_{i+1}" for i in range(n_waves)])

    return pd.concat([df_mean, df_sigma], axis=1)

In [ ]:
model.fit(train, train_labels)

sigma_predictor = SigmaPredictor()
very_bad = np.arange(train_labels.shape[0])[train['very_bad'].values == True]
sigma_predictor.fit(oof_pred, train_labels.values, outliers, very_bad)

In [ ]:
def bootstrap_uncertainty_inference( 
    X_train,
    y_train,
    X_test,
    n_bootstraps = 100, 
    random_state = 42,
):
    """
    Sigma estimation via bootstrapping
    """
    if random_state is not None:
        np.random.seed(random_state)
    
    y_train_values = y_train
        
    n_test_samples = X_test.shape[0]
    n_targets = y_train_values.shape[1]
    
    predictions = np.full((n_test_samples, n_targets, n_bootstraps), np.nan)
    
    bootstrap_iter = range(n_bootstraps)
    bootstrap_iter = tqdm(bootstrap_iter, desc="bootstrap interations")
    
    for b in bootstrap_iter:
        bootstrap_indices = np.random.choice(
            len(X_train), size=len(X_train), replace=True
        )
        
        X_bootstrap = X_train.iloc[bootstrap_indices].reset_index(drop=True)
        y_bootstrap = y_train_values.iloc[bootstrap_indices].reset_index(drop=True)
        
        model_bootstrap = CustomRidge()
        model_bootstrap.fit(X_bootstrap, y_bootstrap)
        
        y_pred = model_bootstrap.predict(X_test)   
        predictions[:, :, b] = y_pred
     
    return predictions

In [ ]:
import pickle
import gc
import pandas as pd
import numpy as np

# Load test data
test_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2025/test_star_info.csv', index_col='planet_id')
sample_submission = pd.read_csv('/kaggle/input/ariel-data-challenge-2025/sample_submission.csv', index_col='planet_id')
wavelengths = pd.read_csv('/kaggle/input/ariel-data-challenge-2025/wavelengths.csv')
test_star_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2025/test_star_info.csv')

# Clean up training data from memory
# del data_train
gc.collect()

# Run preprocessing
os.environ["PREPROCESS_MODE"] = "test"
!python preprocess.py
!rm -rf *AIRS-CH0_signal*

# Load preprocessed test signal
data_test = np.load("signal_v2.npy")

# Load the configuration fitted on training data
config = FeatureEngineeringConfig.load('feature_config.pkl')

# Generate features for test data
# ✅ FIXED: Use test_star_info DataFrame, not a string path
outliers, test_features, _ = feature_engineering(
    test_star_info,  # ✅ This is the DataFrame, don't overwrite it!
    data_test, 
    config=config,
    is_training=False
)

# Get indices of outliers and very bad samples
outliers = np.arange(test_features.shape[0])[outliers]
very_bad = np.arange(test_features.shape[0])[test_features['very_bad'].values == True]

# Generate predictions
test_pred = model.predict(test_features)

# Bootstrap uncertainty estimation
boot_pred = bootstrap_uncertainty_inference(train, train_labels, test_features, n_bootstraps=1000)
test_bootstrap_preds = boot_pred.std(-1) * 2.75

# Clip predictions to valid physical range
test_pred = np.maximum(test_pred, 0.003)
test_pred = np.minimum(test_pred, 0.1)

print('sigma', sigma_pred)

def postprocessing(pred_array, index, sigma_pred, sigma_predictor, outliers, very_bad, bootstrap_preds=None, column_names=None):
    """
    Convert predictions and uncertainty into final submission DataFrame
    """
    sigma_array = sigma_predictor.predict(np.zeros_like(pred_array), pred_array, outliers, very_bad, bootstrap_preds=bootstrap_preds)
    df_pred = pd.DataFrame(pred_array.clip(0, None), index=index, columns=column_names)
    df_sigma = pd.DataFrame(sigma_array, index=index, columns=[f"sigma_{i}" for i in range(1, len(column_names)+1)])
    return pd.concat([df_pred, df_sigma], axis=1)

# Create submission
submission_df = postprocessing(
    pred_array=test_pred,
    index=sample_submission.index,
    sigma_pred=sigma_pred,
    sigma_predictor=sigma_predictor,
    column_names=wavelengths.columns,
    bootstrap_preds=test_bootstrap_preds,
    outliers=outliers,
    very_bad=very_bad
)

# Save submission
submission_df.to_csv('submission.csv')
!head submission.csv

In [ ]:
import joblib
import os

# after training
os.makedirs("/kaggle/working/models", exist_ok=True)
joblib.dump(model, "/kaggle/working/models/custom_ridge.pkl")

print("✅ Model saved at /kaggle/working/models/custom_ridge.pkl")


os.makedirs("/kaggle/working/models", exist_ok=True)
joblib.dump(sigma_predictor, "/kaggle/working/models/SigmaPredictor.pkl")

print("✅ sigma_predictor saved at /kaggle/working/models/SigmaPredictor.pkl")